In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

In [2]:
def preprocZ(ds):
     
    # Not sure how to pass this to a preproc function
    level=850
    lons=[240,360]
    lats=[0,60]
    v='z'
    
    # Extract region and level
    ds=ds.sel(isobaricInhPa=level,latitude=slice(lats[1],lats[0]),longitude=slice(lons[0],lons[1]))

    # Create a new xarray dataset
    ds_tmp=xr.DataArray(ds[v],
                        coords={'lat':ds['latitude'].values,
                                'lon': ds['longitude'].values},
                            dims=['lat','lon'])   
    ds_tmp=ds_tmp.to_dataset(name=v+str(int(level)))
    
    return ds_tmp

In [3]:
def preprocU(ds):
     
    # Not sure how to pass this to a preproc function
    level=850
    lons=[240,360]
    lats=[0,60]
    v='u'
    
    # Extract region and level
    ds=ds.sel(isobaricInhPa=level,latitude=slice(lats[1],lats[0]),longitude=slice(lons[0],lons[1]))

    # Create a new xarray dataset
    ds_tmp=xr.DataArray(ds[v],
                        coords={'lat':ds['latitude'].values,
                                'lon': ds['longitude'].values},
                            dims=['lat','lon'])   
    ds_tmp=ds_tmp.to_dataset(name=v+str(int(level)))
    
    return ds_tmp

In [4]:
# Dates
sdate='19790101'
edate='20191231'

# Years List
yrs_list=np.arange(1979,2019)

In [5]:
# Input
path='/shared/working/rean/era-interim/daily/data/'
fname='ei.oper.an.pl.regn128cm.'

# Output
out_path='/project/predictability/kpegion/wxregimes/era-interim/'
ofname='erai.z850_u850_nash_1979-2015.nc'
outfile=out_path+ofname

In [6]:
# Create empty list to append data for each month
ds_Z_months=[]
ds_U_months=[]

# Loop over months, get list of files and read in data for each month
for m in np.arange(1,13):
        
    # Get all the filenames for this month for all years
    fnamesZ = [f"{path}{year}/{fname}{year}{m:02d}*" for year in yrs_list]       
    print(fnamesZ)
    # Create list of all filenames for this month, and all years
    filesZ=[]
    for files in fnamesZ:
        f2=glob.glob(files)
        for f in f2:
            filesZ.append(f)
    
    filesZ=sorted(filesZ)
    
    # Read Geopotential Height (Z) data           
    ds_Z=xr.open_mfdataset(filesZ,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': 'Geopotential'}},
                           preprocess=preprocZ)
 
    # Read zonal wind (U) data
    ds_U=xr.open_mfdataset(filesZ,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': 'U component of wind'}},
                           preprocess=preprocU) 

    
    # Create dates for time assign to time dimension
    dates_all=pd.date_range(start=sdate,end=edate,freq='D')
    print(dates_all)
    seas_dates=dates_all[(dates_all.month==m)]
    print(seas_dates)
    ds_Z['time']=seas_dates
    ds_U['time']=seas_dates
        
    # Append the latest month to the list
    ds_Z_months.append(ds_Z)
    ds_U_months.append(ds_U)
        
# Combine the months into the init dimension
ds_Z_months = xr.combine_nested(ds_Z_months, concat_dim='time')
ds_U_months = xr.combine_nested(ds_U_months, concat_dim='time')

['/shared/working/rean/era-interim/daily/data/1997/ei.oper.an.pl.regn128cm.199701*', '/shared/working/rean/era-interim/daily/data/1998/ei.oper.an.pl.regn128cm.199801*', '/shared/working/rean/era-interim/daily/data/1999/ei.oper.an.pl.regn128cm.199901*', '/shared/working/rean/era-interim/daily/data/2000/ei.oper.an.pl.regn128cm.200001*', '/shared/working/rean/era-interim/daily/data/2001/ei.oper.an.pl.regn128cm.200101*', '/shared/working/rean/era-interim/daily/data/2002/ei.oper.an.pl.regn128cm.200201*', '/shared/working/rean/era-interim/daily/data/2003/ei.oper.an.pl.regn128cm.200301*', '/shared/working/rean/era-interim/daily/data/2004/ei.oper.an.pl.regn128cm.200401*', '/shared/working/rean/era-interim/daily/data/2005/ei.oper.an.pl.regn128cm.200501*', '/shared/working/rean/era-interim/daily/data/2006/ei.oper.an.pl.regn128cm.200601*', '/shared/working/rean/era-interim/daily/data/2007/ei.oper.an.pl.regn128cm.200701*', '/shared/working/rean/era-interim/daily/data/2008/ei.oper.an.pl.regn128cm.2

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-805d797156aa>", line 25, in <module>
    preprocess=preprocZ)
  File "/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/xarray/backends/api.py", line 912, in open_mfdataset
    datasets = [open_(p, **open_kwargs) for p in paths]
  File "/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/xarray/backends/api.py", line 912, in <listcomp>
    datasets = [open_(p, **open_kwargs) for p in paths]
  File "/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/xarray/backends/api.py", line 500, in open_dataset
    **kwargs,
  File "/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/cfgrib/xarray_plugin.py", line 108, in open_dataset
    errors=errors,
  File "/homes/kpegion/.conda/envs/ml-precip/lib/py

TypeError: object of type 'NoneType' has no len()

In [ ]:
ds_ZU=xr.merge([ds_Z_months,ds_U_months])
ds_ZU

In [ ]:
ds_ZU.to_netcdf(outfile)